# Topic Modeling

## Introduction

Another popular text analysis technique is called topic modeling. The ultimate goal of topic modeling is to find various topics that are present in your corpus. Each document in the corpus will be made up of at least one topic, if not multiple topics.

In this notebook, we will be covering the steps on how to do **Latent Dirichlet Allocation (LDA)**, which is one of many topic modeling techniques. It was specifically designed for text data.

To use a topic modeling technique, you need to provide (1) a document-term matrix and (2) the number of topics you would like the algorithm to pick up.

Once the topic modeling technique is applied, your job as a human is to interpret the results and see if the mix of words in each topic make sense. If they don't make sense, you can try changing up the number of topics, the terms in the document-term matrix, model parameters, or even try a different model.

## Topic Modeling - Attempt #1 (All Text)

In [ ]:
# Let's read in our document-term matrix
import pandas as pd
import pickle

data = pd.read_pickle('dtm(comedians).pkl')
data

,aa,aaaaah,aaah,aah,abandon,abandoned,abbott,abc,abcdefg,abcs,...,zone,zoning,zoo,zoom,zoomed,zubak,zuckerberg,zuzus,éclair,éponine
ali,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
anthony,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
bill,0,1,0,0,0,0,0,0,0,1,...,0,1,0,0,0,0,0,0,0,0
bo,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
dave,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
david,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
george,0,0,0,0,2,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
hasan,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
jacqueline,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,1
jeff,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [ ]:
# Import the necessary modules for LDA with gensim
# Terminal / Anaconda Navigator: conda install -c conda-forge gensim
from gensim import matutils, models
import scipy.sparse

# import logging
# logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [ ]:
# One of the required inputs is a term-document matrix
tdm = data.transpose()
tdm.head()

,ali,anthony,bill,bo,dave,david,george,hasan,jacqueline,jeff,...,jim,jimmy,john,kelsey,kevin,louis,mike,ramy,ricky,taylor
aa,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
aaaaah,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
aaah,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
aah,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,3,0,0,0,0
abandon,0,0,0,0,0,0,2,0,0,0,...,0,0,0,0,3,0,0,0,0,0


In [ ]:
# We're going to put the term-document matrix into a new gensim format, from df --> sparse matrix --> gensim corpus
sparse_counts = scipy.sparse.csr_matrix(tdm)
corpus = matutils.Sparse2Corpus(sparse_counts)
print(corpus)

In [ ]:
# Gensim also requires dictionary of the all terms and their respective location in the term-document matrix
cv = pickle.load(open("cv(comedians).pkl", "rb"))
id2word = dict((v, k) for k, v in cv.vocabulary_.items())

Now that we have the corpus (term-document matrix) and id2word (dictionary of location: term), we need to specify two other parameters - the number of topics and the number of passes. Let's start the number of topics at 2, see if the results make sense, and increase the number from there.

In [ ]:
# Now that we have the corpus (term-document matrix) and id2word (dictionary of location: term),
# we need to specify two other parameters as well - the number of topics and the number of passes
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=2, passes=10)
lda.print_topics()

[(0,
  '0.034*"like" + 0.015*"just" + 0.015*"know" + 0.015*"im" + 0.012*"dont" + 0.010*"right" + 0.008*"thats" + 0.007*"youre" + 0.006*"got" + 0.006*"fucking"'),
 (1,
  '0.016*"know" + 0.011*"like" + 0.011*"just" + 0.009*"im" + 0.007*"dont" + 0.005*"thats" + 0.004*"youre" + 0.004*"did" + 0.004*"think" + 0.004*"way"')]

In [ ]:
# LDA for num_topics = 3
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=3, passes=10)
lda.print_topics()

[(0,
  '0.015*"know" + 0.014*"like" + 0.012*"right" + 0.010*"just" + 0.009*"im" + 0.007*"dont" + 0.007*"thats" + 0.006*"yeah" + 0.005*"fucking" + 0.005*"said"'),
 (1,
  '0.041*"like" + 0.018*"know" + 0.017*"just" + 0.014*"im" + 0.011*"dont" + 0.007*"youre" + 0.007*"thats" + 0.005*"right" + 0.005*"people" + 0.005*"think"'),
 (2,
  '0.016*"like" + 0.013*"im" + 0.013*"just" + 0.012*"dont" + 0.011*"know" + 0.011*"right" + 0.008*"fucking" + 0.008*"thats" + 0.007*"got" + 0.006*"youre"')]

In [ ]:
# LDA for num_topics = 4
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=4, passes=10)
lda.print_topics()

[(0,
  '0.025*"like" + 0.015*"just" + 0.015*"know" + 0.014*"im" + 0.013*"dont" + 0.011*"right" + 0.008*"thats" + 0.008*"youre" + 0.007*"got" + 0.007*"said"'),
 (1,
  '0.001*"like" + 0.001*"know" + 0.001*"im" + 0.001*"just" + 0.000*"dont" + 0.000*"right" + 0.000*"thats" + 0.000*"oh" + 0.000*"said" + 0.000*"fucking"'),
 (2,
  '0.034*"like" + 0.018*"know" + 0.016*"just" + 0.014*"im" + 0.010*"dont" + 0.007*"thats" + 0.007*"right" + 0.007*"youre" + 0.005*"think" + 0.005*"oh"'),
 (3,
  '0.003*"like" + 0.002*"ramy" + 0.002*"joe" + 0.002*"im" + 0.001*"know" + 0.001*"youssef" + 0.001*"pray" + 0.001*"ramadan" + 0.001*"just" + 0.001*"dont"')]

These topics aren't looking too great. We've tried modifying our parameters. Let's try modifying our terms list as well.

## Topic Modeling - Attempt #2 (Nouns Only)

One popular trick is to look only at terms that are from one part of speech (only nouns, only adjectives, etc.). Check out the UPenn tag set: https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html.

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# Let's create a function to pull out nouns from a string of text

from nltk import word_tokenize, pos_tag

def nouns(text):
    '''Given a string of text, tokenize the text and pull out only the nouns.'''
    is_noun = lambda pos: pos[:2] == 'NN'
    tokenized = word_tokenize(text)
    all_nouns = [word for (word, pos) in pos_tag(tokenized) if is_noun(pos)]
    return ' '.join(all_nouns)

In [ ]:
# Read in the cleaned data, before the CountVectorizer step
data_clean = pd.read_pickle('data_clean(comedians).pkl')
data_clean

,transcript,full_name
ali,ali wong baby cobra transcript scraps from...,Ali Wong
anthony,anthony jeselnik thoughts and prayers full ...,Anthony Jeselnik
bill,bill burr im sorry you feel that way full t...,Bill Burr
bo,page not found scraps from the loft \r\t\ts...,Bo Burnham
dave,dave chappelle the age of spin transcript ...,Ceorge Carlin
david,david cross worst daddy in the world transc...,Dave Chappelle
george,george carlin playboy interview scraps from...,David Cross
hasan,page not found scraps from the loft \r\t\ts...,Hasan Minhaj
jacqueline,jacqueline novak get on your knees transcri...,Jacqueline Novak
jeff,jeff dunham im with cupid transcript scrap...,Jeff Dunham


In [ ]:
# Apply the nouns function to the transcripts to filter only on nouns
#nltk-->'punkt'
data_nouns = pd.DataFrame(data_clean.transcript.apply(nouns))
data_nouns

,transcript
ali,ali baby cobra transcript scraps skip reviewsm...
anthony,anthony jeselnik thoughts prayers transcript s...
bill,bill sorry way transcript scraps skip reviewsm...
bo,page scraps skip reviewsmovie transcriptsstanl...
dave,chappelle age spin transcript scraps skip revi...
david,david cross daddy world transcript scraps skip...
george,george carlin playboy interview scraps skip re...
hasan,page scraps skip reviewsmovie transcriptsstanl...
jacqueline,jacqueline knees scraps skip reviewsmovie tran...
jeff,jeff dunham im transcript scraps skip reviewsm...


In [ ]:
# Create a new document-term matrix using only nouns
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords

# Re-add the additional stop words since we are recreating the document-term matrix
add_stop_words = ['like', 'im', 'know', 'just', 'dont', 'thats', 'right', 'people',
                  'youre', 'got', 'gonna', 'time', 'think', 'yeah', 'said', 'and',
                  'or', 'a', 'the', 'for', 'you']
stop_words = text.ENGLISH_STOP_WORDS.union(add_stop_words)
#stop_words = stopwords.words('english')
# Recreate a document-term matrix with only nouns
cvn = CountVectorizer(stop_words=list(stop_words))
data_cvn = cvn.fit_transform(data_nouns.transcript)
data_dtmn = pd.DataFrame(data_cvn.toarray(), columns=cvn.get_feature_names_out())
data_dtmn.index = data_nouns.index
data_dtmn

,aa,aah,abandon,abc,abcdefg,abcs,abdomen,abercrombie,abigail,ability,...,zipper,zombie,zombies,zone,zoo,zoom,zubak,zuckerberg,zuzus,éclair
ali,0,0,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
anthony,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
bill,0,0,0,0,0,1,0,0,0,0,...,0,1,1,0,0,0,0,0,0,0
bo,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
dave,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
david,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
george,0,0,1,0,0,0,0,0,0,5,...,0,0,0,0,0,0,0,0,0,0
hasan,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
jacqueline,0,0,0,0,0,0,1,2,1,1,...,1,0,0,0,0,0,0,0,1,0
jeff,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [ ]:
# # Recreate a document-term matrix with only nouns
# cvn = CountVectorizer(stop_words=stop_words)
# data_cvn = cvn.fit_transform(data_nouns['transcript'])
# data_dtmn = pd.DataFrame(data_cvn.toarray(), columns=cvn.get_feature_names_out())
# data_dtmn.index = data_nouns.index
# data_dtmn

In [ ]:
# Create the gensim corpus
corpusn = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmn.transpose()))

# Create the vocabulary dictionary
id2wordn = dict((v, k) for k, v in cvn.vocabulary_.items())

In [ ]:
# Let's start with 2 topics
ldan = models.LdaModel(corpus=corpusn, num_topics=2, id2word=id2wordn, passes=10)
ldan.print_topics()

[(0,
  '0.008*"way" + 0.007*"thing" + 0.007*"life" + 0.005*"cause" + 0.005*"job" + 0.005*"gon" + 0.005*"hes" + 0.004*"lot" + 0.004*"blow" + 0.004*"kind"'),
 (1,
  '0.009*"man" + 0.008*"day" + 0.007*"shit" + 0.006*"thing" + 0.005*"fuck" + 0.005*"years" + 0.005*"life" + 0.005*"way" + 0.005*"hes" + 0.005*"guy"')]

In [ ]:
# Let's try topics = 3
ldan = models.LdaModel(corpus=corpusn, num_topics=3, id2word=id2wordn, passes=10)
ldan.print_topics()

[(0,
  '0.010*"man" + 0.009*"shit" + 0.008*"life" + 0.008*"thing" + 0.008*"gon" + 0.007*"hes" + 0.006*"cause" + 0.005*"fuck" + 0.005*"way" + 0.005*"lot"'),
 (1,
  '0.007*"day" + 0.006*"man" + 0.005*"thing" + 0.005*"years" + 0.005*"life" + 0.005*"bit" + 0.005*"hes" + 0.004*"way" + 0.004*"id" + 0.004*"things"'),
 (2,
  '0.008*"way" + 0.007*"thing" + 0.006*"day" + 0.006*"job" + 0.005*"cause" + 0.005*"life" + 0.005*"lot" + 0.005*"things" + 0.005*"blow" + 0.005*"penis"')]

In [ ]:
# Let's try 4 topics
ldan = models.LdaModel(corpus=corpusn, num_topics=4, id2word=id2wordn, passes=10)
ldan.print_topics()

[(0,
  '0.007*"man" + 0.005*"life" + 0.005*"school" + 0.005*"years" + 0.005*"lot" + 0.004*"way" + 0.004*"bit" + 0.004*"id" + 0.004*"friends" + 0.004*"day"'),
 (1,
  '0.009*"thing" + 0.008*"way" + 0.008*"life" + 0.007*"cause" + 0.007*"lot" + 0.006*"shit" + 0.006*"man" + 0.006*"day" + 0.006*"women" + 0.006*"hes"'),
 (2,
  '0.008*"day" + 0.008*"joke" + 0.007*"thing" + 0.005*"things" + 0.005*"comedy" + 0.005*"years" + 0.005*"transcriptsgeorge" + 0.005*"way" + 0.005*"hes" + 0.004*"shes"'),
 (3,
  '0.014*"man" + 0.009*"shit" + 0.007*"fuck" + 0.007*"day" + 0.006*"gon" + 0.006*"thing" + 0.006*"hes" + 0.006*"guy" + 0.006*"cause" + 0.005*"way"')]

## Topic Modeling - Attempt #3 (Nouns and Adjectives)

In [ ]:
# Let's create a function to pull out nouns from a string of text
def nouns_adj(text):
    '''Given a string of text, tokenize the text and pull out only the nouns and adjectives.'''
    is_noun_adj = lambda pos: pos[:2] == 'NN' or pos[:2] == 'JJ'
    tokenized = word_tokenize(text)
    nouns_adj = [word for (word, pos) in pos_tag(tokenized) if is_noun_adj(pos)]
    return ' '.join(nouns_adj)

In [ ]:
# Apply the nouns function to the transcripts to filter only on nouns
data_nouns_adj = pd.DataFrame(data_clean.transcript.apply(nouns_adj))
data_nouns_adj

,transcript
ali,ali wong baby cobra transcript scraps loft ski...
anthony,anthony jeselnik thoughts prayers full transcr...
bill,bill im sorry way full transcript scraps loft ...
bo,page scraps loft skip moviesmovie reviewsmovie...
dave,chappelle age spin transcript scraps loft skip...
david,david cross worst daddy world transcript scrap...
george,george carlin playboy interview scraps loft sk...
hasan,page scraps loft skip moviesmovie reviewsmovie...
jacqueline,jacqueline novak knees scraps loft skip movies...
jeff,jeff dunham im cupid transcript scraps loft sk...


In [ ]:
# Create a new document-term matrix using only nouns and adjectives, also remove common words with max_df
cvna = CountVectorizer(stop_words=list(stop_words), max_df=.8)
data_cvna = cvna.fit_transform(data_nouns_adj.transcript)
data_dtmna = pd.DataFrame(data_cvna.toarray(), columns=cvna.get_feature_names_out())
data_dtmna.index = data_nouns_adj.index
data_dtmna

,aa,aaaaah,aah,abandon,abc,abcdefg,abcs,abdomen,abercrombie,abigail,...,zipper,zombie,zombies,zone,zoo,zoom,zubak,zuckerberg,zuzus,éclair
ali,0,0,0,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
anthony,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
bill,0,1,0,0,0,0,1,0,0,0,...,0,1,1,0,0,0,0,0,0,0
bo,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
dave,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
david,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
george,0,0,0,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
hasan,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
jacqueline,0,0,0,0,0,0,0,1,2,1,...,1,0,0,0,0,0,0,0,2,0
jeff,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [ ]:
# Create the gensim corpus
corpusna = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmna.transpose()))

# Create the vocabulary dictionary
id2wordna = dict((v, k) for k, v in cvna.vocabulary_.items())

In [ ]:
# Let's start with 2 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=2, id2word=id2wordna, passes=10)
ldana.print_topics()

[(0,
  '0.004*"job" + 0.004*"blow" + 0.004*"oh" + 0.004*"penis" + 0.003*"cause" + 0.003*"bit" + 0.003*"school" + 0.003*"wife" + 0.003*"old" + 0.003*"year"'),
 (1,
  '0.008*"shit" + 0.006*"fuck" + 0.005*"um" + 0.005*"cause" + 0.004*"old" + 0.004*"id" + 0.004*"women" + 0.004*"oh" + 0.003*"kids" + 0.003*"joke"')]

In [ ]:
# Let's try 3 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=3, id2word=id2wordna, passes=10)
ldana.print_topics()

[(0,
  '0.005*"cause" + 0.005*"job" + 0.004*"old" + 0.004*"bit" + 0.004*"oh" + 0.004*"penis" + 0.004*"blow" + 0.004*"kid" + 0.004*"id" + 0.003*"school"'),
 (1,
  '0.005*"id" + 0.003*"course" + 0.003*"year" + 0.002*"heart" + 0.002*"school" + 0.002*"coke" + 0.002*"music" + 0.002*"love" + 0.002*"tonight" + 0.002*"cocaine"'),
 (2,
  '0.012*"shit" + 0.008*"um" + 0.008*"fuck" + 0.006*"cause" + 0.005*"women" + 0.005*"oh" + 0.005*"dude" + 0.004*"black" + 0.004*"old" + 0.004*"everybody"')]

In [ ]:
# Let's try 4 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=4, id2word=id2wordna, passes=10)
ldana.print_topics()

[(0,
  '0.016*"um" + 0.005*"oh" + 0.004*"love" + 0.004*"uh" + 0.003*"okay" + 0.003*"cause" + 0.003*"single" + 0.003*"friend" + 0.003*"parents" + 0.002*"girl"'),
 (1,
  '0.012*"shit" + 0.008*"fuck" + 0.007*"cause" + 0.006*"women" + 0.005*"old" + 0.005*"hey" + 0.004*"kids" + 0.004*"kid" + 0.004*"black" + 0.004*"oh"'),
 (2,
  '0.005*"blow" + 0.005*"job" + 0.005*"penis" + 0.005*"school" + 0.004*"id" + 0.004*"bit" + 0.003*"old" + 0.003*"cause" + 0.003*"year" + 0.003*"sort"'),
 (3,
  '0.005*"id" + 0.005*"joke" + 0.004*"old" + 0.003*"oh" + 0.003*"bad" + 0.003*"women" + 0.003*"hair" + 0.003*"nuts" + 0.003*"rape" + 0.003*"feet"')]

## Identify Topics in Each Document

Out of the 9 topic models we looked at, the nouns and adjectives, 4 topic one made the most sense. So let's pull that down here and run it through some more iterations to get more fine-tuned topics.

In [ ]:
# Our final LDA model (for now)
ldana = models.LdaModel(corpus=corpusna, num_topics=4, id2word=id2wordna, passes=80)
for idx, topic in ldana.print_topics(-1):
    print("Topic {}: {}".format(idx + 1, topic))

Topic 1: 0.006*"called" + 0.005*"fuck" + 0.005*"used" + 0.005*"id" + 0.005*"began" + 0.004*"loved" + 0.004*"help" + 0.003*"doesnt" + 0.003*"talk" + 0.003*"having"
Topic 2: 0.006*"wanted" + 0.006*"knew" + 0.006*"seen" + 0.006*"remember" + 0.006*"having" + 0.005*"cause" + 0.005*"saw" + 0.005*"looked" + 0.005*"happened" + 0.005*"guys"
Topic 3: 0.008*"fuck" + 0.007*"used" + 0.007*"guys" + 0.007*"comes" + 0.007*"happened" + 0.006*"hes" + 0.005*"saw" + 0.005*"watch" + 0.005*"sitting" + 0.004*"hate"
Topic 4: 0.009*"shit" + 0.007*"guys" + 0.006*"seen" + 0.006*"cause" + 0.006*"comes" + 0.006*"looking" + 0.006*"hes" + 0.006*"sleep" + 0.006*"aint" + 0.006*"fuck"


These four topics look pretty decent. Let's settle on these for now.
* Topic 0: awards, entertainment
* Topic 1: Profanity, Social Commentary
* Topic 2: Humor, Sexual References
* Topic 3: Everyday Conversations, Expletives

In [ ]:
# # Let's take a look at which topics each transcript contains
# corpus_transformed = ldana[corpusna]

# # Unpack the values from corpus_transformed
# topics_per_transcript = [[a for (a, b) in doc] for doc in corpus_transformed]

# # Combine with transcript indices
# topics_with_indices = list(zip(topics_per_transcript, data_dtmna.index))

# # Print the result
# for idx, topic in ldana.print_topics(-1):
#      print("Topic {}: {}".format(idx + 1, topic))

Topic 1: 0.010*"remember" + 0.010*"looked" + 0.009*"saw" + 0.008*"walked" + 0.007*"suffer" + 0.007*"started" + 0.007*"chappelle" + 0.006*"called" + 0.006*"knew" + 0.006*"met"
Topic 2: 0.012*"seen" + 0.007*"guys" + 0.007*"kev" + 0.007*"aint" + 0.006*"having" + 0.006*"used" + 0.006*"watching" + 0.006*"looking" + 0.005*"learned" + 0.005*"stay"
Topic 3: 0.008*"guys" + 0.006*"thank" + 0.006*"comes" + 0.006*"understand" + 0.006*"doesnt" + 0.006*"die" + 0.006*"hed" + 0.006*"bring" + 0.005*"theyre" + 0.005*"ill"
Topic 4: 0.008*"called" + 0.007*"fuck" + 0.007*"used" + 0.006*"id" + 0.006*"began" + 0.005*"loved" + 0.005*"doesnt" + 0.004*"having" + 0.004*"thinking" + 0.004*"felt"
Topic 5: 0.007*"cause" + 0.006*"wan" + 0.006*"knew" + 0.006*"wanted" + 0.006*"dating" + 0.005*"talk" + 0.005*"seen" + 0.005*"remember" + 0.005*"play" + 0.005*"walked"
Topic 6: 0.010*"happened" + 0.009*"fuck" + 0.009*"comes" + 0.007*"wan" + 0.007*"used" + 0.007*"die" + 0.006*"hes" + 0.006*"shit" + 0.006*"saw" + 0.006*"does

In [ ]:
# Let's take a look at which topics each transcript contains
corpus_transformed = ldana[corpusna]
list(zip([a for [(a,b)] in corpus_transformed], data_dtmna.index))

[([1], 'ali'), ([2], 'anthony'), ([2], 'bill'), ([0, 1, 2, 3], 'bo'), ([3], 'dave'), ([2], 'david'), ([3], 'george'), ([0, 1, 2, 3], 'hasan'), ([2], 'jacqueline'), ([1], 'jeff'), ([3], 'jenny'), ([3], 'jim'), ([0], 'jimmy'), ([3], 'john'), ([2], 'kelsey'), ([3], 'kevin'), ([1], 'louis'), ([1], 'mike'), ([0], 'ramy'), ([2], 'ricky'), ([1], 'taylor')]


For a first pass of LDA, these kind of make sense to me, so we'll call it a day for now.
* Topic 0: Awards, Entertainment [Jimmy, Ramy]
* Topic 1: Profanity, Social Commentary [Ali, Jeff, Louis, Mike]
* Topic 2:  Humor, Sexual References [Anthony, Bill, David, Jacqueline, Kelsey, Ricky, Taylor]
* Topic 3:  Everyday Conversations, Expletives [Bo, Dave, George, Hasan, Jenny, Jim, John, Kevin]

### Assignment:
1. Try further modifying the parameters of the topic models above and see if you can get better topics.
2. Create a new topic model that includes terms from a different [part of speech](https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html) and see if you can get better topics.

In [ ]:
#Part 1
ldana = models.LdaModel(corpus=corpusna, num_topics=6, id2word=id2wordna, passes=50)
for idx, topic in ldana.print_topics(-1):
     print("Topic {}: {}".format(idx + 1, topic))

Topic 1: 0.010*"remember" + 0.010*"looked" + 0.009*"saw" + 0.008*"walked" + 0.007*"suffer" + 0.007*"started" + 0.007*"chappelle" + 0.006*"called" + 0.006*"knew" + 0.006*"met"
Topic 2: 0.012*"seen" + 0.007*"guys" + 0.007*"kev" + 0.007*"aint" + 0.006*"having" + 0.006*"used" + 0.006*"watching" + 0.006*"looking" + 0.005*"learned" + 0.005*"stay"
Topic 3: 0.008*"guys" + 0.006*"thank" + 0.006*"comes" + 0.006*"understand" + 0.006*"doesnt" + 0.006*"die" + 0.006*"hed" + 0.006*"bring" + 0.005*"theyre" + 0.005*"ill"
Topic 4: 0.008*"called" + 0.007*"fuck" + 0.007*"used" + 0.006*"id" + 0.006*"began" + 0.005*"loved" + 0.005*"doesnt" + 0.004*"having" + 0.004*"thinking" + 0.004*"felt"
Topic 5: 0.007*"cause" + 0.006*"wan" + 0.006*"knew" + 0.006*"wanted" + 0.006*"dating" + 0.005*"talk" + 0.005*"seen" + 0.005*"remember" + 0.005*"play" + 0.005*"walked"
Topic 6: 0.010*"happened" + 0.009*"fuck" + 0.009*"comes" + 0.007*"wan" + 0.007*"used" + 0.007*"die" + 0.006*"hes" + 0.006*"shit" + 0.006*"saw" + 0.006*"does



1.   Topic 1: Memories and Encounters
2.   Topic 2: Social Interactions and Learning
3.   Topic 3: Relationships and Communication
4.   Topic 4: Emotional Expression and Reflection
5.   Topic 5: Desires and Intentions
6.   Topic 6: Unexpected Events and Reactions







In [ ]:
#part 2
# Function to tokenize and filter for verbs
def verb(text):
    '''Given a string of text, tokenize the text and pull out only the verbs.'''
    is_verb = lambda pos: pos[:2] == 'VB'
    tokenized = word_tokenize(text)
    all_verb = [word for (word, pos) in pos_tag(tokenized) if is_verb(pos)]
    return ' '.join(all_verb)

# Apply the nouns function to the transcripts to filter only on nouns
data_verb = pd.DataFrame(data_clean.transcript.apply(verb))
data_verb

# Create a new document-term matrix using only nouns, adjectives, and verbs, also remove common words with max_df
cvna = CountVectorizer(stop_words=list(stop_words), max_df=.8)
data_cvna = cvna.fit_transform(data_verb.transcript)
data_dtmna = pd.DataFrame(data_cvna.toarray(), columns=cvna.get_feature_names_out())
data_dtmna.index = data_verb.index

# Create the gensim corpus
corpus_verb = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmna.transpose()))

# Create the vocabulary dictionary
id2word_verb = dict((v, k) for k, v in cvna.vocabulary_.items())

# Attempt LDA with verbs
lda_verb = models.LdaModel(corpus=corpus_verb, num_topics=4, id2word=id2word_verb, passes=10)
for idx, topic in ldana.print_topics(-1):
    print("Topic {}: {}".format(idx + 1, topic))

Topic 1: 0.006*"called" + 0.005*"fuck" + 0.005*"used" + 0.005*"id" + 0.005*"began" + 0.004*"loved" + 0.004*"help" + 0.003*"doesnt" + 0.003*"talk" + 0.003*"having"
Topic 2: 0.006*"wanted" + 0.006*"knew" + 0.006*"seen" + 0.006*"remember" + 0.006*"having" + 0.005*"cause" + 0.005*"saw" + 0.005*"looked" + 0.005*"happened" + 0.005*"guys"
Topic 3: 0.008*"fuck" + 0.007*"used" + 0.007*"guys" + 0.007*"comes" + 0.007*"happened" + 0.006*"hes" + 0.005*"saw" + 0.005*"watch" + 0.005*"sitting" + 0.004*"hate"
Topic 4: 0.009*"shit" + 0.007*"guys" + 0.006*"seen" + 0.006*"cause" + 0.006*"comes" + 0.006*"looking" + 0.006*"hes" + 0.006*"sleep" + 0.006*"aint" + 0.006*"fuck"


# **Conclusion**

1.	Ali Wong
*	Full Name: Ali Wong
*	Polarity Class: Positive
*	Subjectivity Class: Objective
*	Topic 1: Profanity, Social Commentary

2.	Anthony Jeselnik
8	Full Name: Anthony Jeselnik
*	Polarity Class: Positive
*	Subjectivity Class: Subjective
*	Topic 2: Humor, Sexual References

3.	Bill Burr
*	Full Name: Bill Burr
*	Polarity Class: Positive
*	Subjectivity Class: Subjective
*	Topic 2: Humor, Sexual References

4.	Bo Burnham
*	Full Name: Bo Burnham
*	Polarity Class: Positive
*	Subjectivity Class: Objective
*	Topic 3: Everyday Conversations, Expletives

5.	 George Carlin
*	Full Name: George Carlin
*	Polarity Class: Positive
*	Subjectivity Class: Subjective
*	Topic 3: Everyday Conversations, Expletives

6.	Dave Chappelle
*	Full Name: Dave Chappelle
*	Polarity Class: Positive
*	Subjectivity Class: Subjective
*	Topic 3: Everyday Conversations, Expletives

7.	David Cross
*	Full Name: David Cross
*	Polarity Class: Positive
*	Subjectivity Class: Objective
*	Topic 2: Humor, Sexual References

8.	Hasan Minhaj
*	Full Name: Hasan Minhaj
*	Polarity Class: Positive
*	Subjectivity Class: Objective
*	Topic 3: Everyday Conversations, Expletives

9.	Jacqueline Novak
*	Full Name: Jacqueline Novak
*	Polarity Class: Positive
*	Subjectivity Class: Subjective
*	Topic 2: Humor, Sexual References

10.	Jeff Dunham
*	Full Name: Jeff Dunham
*	Polarity Class: Positive
*	Subjectivity Class: Subjective
*	Topic 1: Profanity, Social Commentary

11.	Jenny Slate
*	Full Name: Jenny Slate
*	Polarity Class: Positive
*	Subjectivity Class: Subjective
*	Topic 3: Everyday Conversations, Expletives

12.	Jim Jefferies
*	Full Name: Jim Jefferies
*	Polarity Class: Positive
*	Subjectivity Class: Subjective
*	Topic 3: Everyday Conversations, Expletives

13.	 Jimmy Kimmel
*	Full Name: Jimmy Kimmel
*	Polarity Class: Positive
*	Subjectivity Class: Subjective
*	Topic 0: Awards, Entertainment

14.	John Mulaney
*	Full Name: John Mulaney
*	Polarity Class: Positive
*	Subjectivity Class: Objective
*	Topic 3: Everyday Conversations, Expletives

15.	Kelsey Cook
*	Full Name: Kelsey Cook
*	Polarity Class: Positive
*	Subjectivity Class: Objective
*	Topic 2: Humor, Sexual References

16.	Kevin Bridges
*	Full Name: Kevin Bridges
*	Polarity Class: Positive
*	Subjectivity Class: Subjective
*	Topic 3: Everyday Conversations, Expletives

17.	Louis C.K.
*	Full Name: Louis C.K.
*	Polarity Class: Positive
*	Subjectivity Class: Subjective
*	Topic 1: Profanity, Social Commentary

18.	Mike Epps
*	Full Name: Mike Epps
*	Polarity Class: Negative
*	Subjectivity Class: Subjective
*	Topic 1: Profanity, Social Commentary

19.	Ramy Youssef
*	Full Name: Ramy Youssef
*	Polarity Class: Positive
*	Subjectivity Class: Objective
*	Topic 0: Awards, Entertainment

20.	Ricky Gervais
*	Full Name: Ricky Gervais
*	Polarity Class: Positive
*	Subjectivity Class: Subjective
*	Topic 2: Humor, Sexual References

21.	Taylor Tomlinson
*	Full Name: Taylor Tomlinson
*	Polarity Class: Positive
*	Subjectivity Class: Objective
*	Topic 2: Humor, Sexual References



1. Ali Wong: Known for profanity and social commentary, she maintains an objective viewpoint in her comedy.

2. Anthony Jeselnik: His humor revolves around sexual references and he tends to express subjective opinions.

3. Bill Burr: Similar to Jeselnik, Burr's comedy centers on humor and sexual references, with a subjective perspective.

4. Bo Burnham: Engages in everyday conversations with expletives but maintains an objective stance in his comedy.

5. George Carlin: Like Burr and Jeselnik, Carlin's comedy involves everyday conversations and expletives, approached subjectively.

6. Dave Chappelle: Shares similarities with Carlin in his focus on everyday conversations and expletives, with a subjective approach.

7. David Cross: He incorporates humor and sexual references into his routines while maintaining an objective viewpoint.

8. Hasan Minhaj: Engages in everyday conversations with expletives but approaches comedy objectively.

9. Jacqueline Novak: Her comedy features humor and sexual references from a subjective standpoint.

10. Jeff Dunham: Mixes profanity with social commentary, expressing subjective opinions.

11. Jenny Slate: Similar to Novak, Slate's comedy involves everyday conversations and expletives, approached subjectively.

12. Jim Jefferies: His comedy revolves around everyday conversations and expletives with a subjective perspective.

13. Jimmy Kimmel: Focuses on awards and entertainment with subjective viewpoints.

14. John Mulaney: Engages in everyday conversations with expletives but maintains an objective stance in his comedy.

15. Kelsey Cook: Incorporates humor and sexual references into her routines while maintaining an objective viewpoint.

16. Kevin Bridges: His comedy involves everyday conversations and expletives approached subjectively.

17. Louis C.K.: Known for profanity and social commentary, he expresses subjective opinions.

18. Mike Epps: Engages in profanity and social commentary with a negative polarity and subjective perspective.

19. Ramy Youssef: Focuses on awards and entertainment from an objective viewpoint.

20. Ricky Gervais: His comedy revolves around humor and sexual references with subjective viewpoints.

21. Taylor Tomlinson: Incorporates humor and sexual references into her routines while maintaining an objective viewpoint.